In [ ]:
from htmd import *
htmd.config(viewer='webgl')

# Building Barnase - Barstar for protein-protein interactions

![](http://pub.htmd.org/73hboiwia98hdj209jq0/barnasebarstar.png)

## Download the two proteins and view them

You can look at their PDB information here and find out their PDB IDs. Then download them using those IDs.

* [Barnase](http://www.rcsb.org/pdb/explore.do?structureId=2f4y)
* [Barstar](http://www.rcsb.org/pdb/explore.do?structureId=2hxx)

You will need to create Molecule objects. Check the documentation on the [Molecule](https://www.htmd.org/docs/htmd.molecule.molecule.html) class.

In [ ]:
barnase = Molecule('2f4y')
barstar = Molecule('2hxx')

## Filter the structures to keep only one chain of each

Pick only one chain of each structure. This will keep the crystal waters corresponding to the chosen chain.

In [ ]:
barnase.filter('chain A')
barstar.filter('chain A')

## Visualize the filtered structures

In [ ]:
from ipywidgets.widgets import Box; w = []
w.append(barnase.view())
w.append(barstar.view())
Box(children=(w[0],w[1]))

## Mutate modified residues

[Barstar](http://www.rcsb.org/pdb/explore/explore.do?structureId=2hxx) has a modified residue for which we lack the parametrization (check under "Small Molecules" on PDB). Mutate the modified Tryptophan in Barstar (resname 4IN) to a normal Tryptophan (TRP):

In [ ]:
barstar.mutateResidue('resname 4IN', 'TRP')

## Assignments and renaming

Assign a different chain (A, B) and segment to each protein (BRN, STR):

In [ ]:
barnase.set('chain', 'A', 'protein')
barstar.set('chain', 'B', 'protein')
barnase.set('segid', 'BRN')
barstar.set('segid', 'STR')

Since the crystal waters were kept, assign them to segid W1 and W2 (assigning waters of both molecules to the same segid can cause problems as they will have same resids)

In [ ]:
barnase.set('segid', 'W1', 'water')
barstar.set('segid', 'W2', 'water')

## Combine the proteins and center them

Create a new molecule which will contain both other molecules.

In [ ]:
mol = Molecule()
mol.append(barnase)
mol.append(barstar)

Now center the new combined molecule on the origin.

In [ ]:
mol.center()

## Solvate the combined system

Find the maximum distance of the atoms from the center:

In [ ]:
from htmd.molecule.util import maxDistance
D = maxDistance(mol); print(D)

Add 5 &Aring; to this distance, to add some extra space in the box, and then solvate (no need to add a salt concentration):

In [ ]:
D += 5
smol = solvate(mol, minmax=[[-D, -D, -D],[D, D, D]])

## Build the solvated system in CHARMM

In [ ]:
molbuilt = charmm.build(smol, outdir='./build/')

## Visualize the built system

Can take a while to load...

In [ ]:
molbuilt.view(sel='protein',style='NewCartoon',
              color='Secondary Structure', hold=True)
molbuilt.view(sel='water',style='lines')